# Event sourcing

Instead of writing directly to the tables, the client writes to an extra table, the `eventstore`, to keep track of all the changes. This allows now only to keep track of the changes, but also allows to revert the database to what it was at any point of time.

Here is what it looks like:

In [1]:
import mdb

client = mdb.MDBClient(hostname='localhost',
                       username='postgres',
                       password='',
                       database='mdb')

es = client.get('eventstore')
es

/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:3010: SAWarning: Did not recognize type 'xml' of column 'xdl'
  "Did not recognize type '%s' of column '%s'" % (attype, name)
/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.conformer, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.software, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
/Users/tga/miniconda3/envs/mdb/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This

,type,id,uuid,data,event,timestamp
0,molecule,23,391acc86-63db-4a43-a420-d605856d79ff,"{'smiles': 'DEF', 'molecule_type_id': '61c3157...",create,2020-03-31 14:08:00.915527
1,xy_data,22,f3393a30-47de-48ee-80e1-c49038984bf1,"{'x': [0.0, 0.10101010101010101, 0.20202020202...",create,2020-03-31 14:07:46.239803
2,data_unit,21,f31f93bf-7c8b-4879-8712-4f63ee266898,{'name': 'stupid_units'},create,2020-03-31 14:07:46.174190
3,experiment,20,8a6ad20d-7128-46dc-8f9f-74bd8889d314,"{'notes': '', 'metadata': {'param1': 100, 'kno...",create,2020-03-31 14:07:44.005201
4,experiment_machine,19,bd6680f3-f6e9-43da-aaae-818fa0d39af8,"{'make': 'Super NMR', 'name': 'Mad NMR', 'mode...",create,2020-03-31 14:07:43.963820
5,experiment_type,18,80d674d7-b6cf-41cb-8244-610121c1ae9b,{'name': 'NMR'},create,2020-03-31 14:07:42.690698
6,synthesis,17,7addb20e-a8f4-4ebb-91bc-d01582b28e29,{'notes': ' # Synthesis of ABC Everything wen...,update,2020-03-31 14:07:38.725166
7,synthesis,16,7addb20e-a8f4-4ebb-91bc-d01582b28e29,"{'hid': 'MAD_2020-03-31_0', 'xdl': '<xdl><reci...",create,2020-03-31 14:07:31.543941
8,molecule,14,b209c5b1-ea69-47ef-8df0-5f7de2f710f2,"{'cas': '13283-31-3', 'cid': 6331.0, 'inchi': ...",update,2020-03-31 14:07:31.440667
9,molecule,15,172cb0af-7b94-406a-9197-91aa1f40a74b,"{'cas': None, 'cid': None, 'inchi': None, 'smi...",update,2020-03-31 14:07:31.440667


## Rollback

To revert at a specific point of time, the client implements a method `rollback`. 

Here we will revert the database to everything before the timestamp `2020-02-17 00:21:59.810407` (creation of data_unit).

In [2]:
client.rollback(before=es.at[3, 'timestamp'].to_pydatetime())

In [3]:
client.get('eventstore').sort_values(by=['id'])

,event,data,id,type,uuid,timestamp
61,create,"{'name': 'Mad Lab', 'short_name': 'MAD'}",1,lab,2b8e9ae2-a5ce-4075-af85-8fdcedbd27fd,2020-03-31 14:05:51.974644
60,create,"{'make': 'MadChem', 'name': 'Mad Machine Doing...",2,synthesis_machine,02e44866-53a9-401d-a6d8-24f1bb52e109,2020-03-31 14:05:53.373362
59,create,{'name': 'fragment'},3,molecule_type,8399fe19-2523-4b86-b504-3df5bdf39c9b,2020-03-31 14:05:55.728723
58,create,"{'smiles': 'A', 'molecule_type_id': '8399fe19-...",4,molecule,172cb0af-7b94-406a-9197-91aa1f40a74b,2020-03-31 14:05:55.782201
57,create,"{'cas': '13283-31-3', 'cid': 6331, 'inchi': 'I...",5,molecule,b209c5b1-ea69-47ef-8df0-5f7de2f710f2,2020-03-31 14:06:01.454134
...,...,...,...,...,...,...
19,create,"{'hid': 'MAD_2020-03-31_0', 'xdl': '<xdl><reci...",60,synthesis,7addb20e-a8f4-4ebb-91bc-d01582b28e29,2020-03-31 14:11:11.058530
20,update,{'notes': ' # Synthesis of ABC Everything wen...,61,synthesis,7addb20e-a8f4-4ebb-91bc-d01582b28e29,2020-03-31 14:11:11.058530
21,create,{'name': 'NMR'},62,experiment_type,80d674d7-b6cf-41cb-8244-610121c1ae9b,2020-03-31 14:11:11.058530
22,create,"{'make': 'Super NMR', 'name': 'Mad NMR', 'mode...",63,experiment_machine,bd6680f3-f6e9-43da-aaae-818fa0d39af8,2020-03-31 14:11:11.058530


We can see here from the event `rollback` everything is deleted and then the event are sequentially repeated until the we reach the given timestamp.